<a href="https://colab.research.google.com/github/saisravan-kodityala/lex_and_yacc/blob/main/LEX_and_YACC_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEX and YACC Compiler in Colab

Drawbacks:
* Regular interrupts (Ctrl+D, Ctrl+C) for shell won't work in Colab while inputting for program.
<br>Workaround: Store your inputs in a txt file and pass it to the program.

In [ ]:
#@title Install *prerqeuisites* (run this cell first to work on LEX/YACC)
!sudo apt install flex bison

## Lex only

In [ ]:
#@title Writing Lex program
%%writefile program.l

%{
    #include <stdio.h>
    int ctChar=0;
    int ctSpace=0;
    int ctWord=0;
    int ctLine=0;
%}
WORD [^ \t\n,\.:]+
EOL [\n]
BLANK [ ]
%%

{WORD} {ctWord++; ctChar+=yyleng;}
{BLANK} {ctSpace++;}
{EOL} {ctLine++;}
. {ctChar++;}
%%

void main(int argc, char *argv[]){
    if(argc!=2){
        printf("Usage:\n\t./a.out <FILENAME>\n");
        exit(0);
    }

    yyin=fopen(argv[1],"r");
    yylex();

    printf("Word Count: %d\n",ctWord);
    printf("Character Count: %d\n",ctChar);
    printf("Space Count: %d\n",ctSpace);
    printf("Line Count: %d\n",ctLine);
    fclose(yyin);

}

int yywrap(){
    return 1;
}

Overwriting program.l


if you want to use at txt as an input

In [ ]:
%%writefile program.txt

This is a sample file.

Writing program.txt


In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

lex -l program.l
gcc lex.yy.c
./a.out program.txt

Word Count: 5
Character Count: 18
Space Count: 4
Line Count: 2


In [9]:
%%writefile branch.l
%{
  #include "y.tab.h"
  #include<stdio.h>
%}
letter [a-zA-Z]
digit [0-9]
%%

"if" return IF;
"else" return ELSE;
"switch" return SWITCH;
"case" return CASE;
"default" return DEFAULT;
"break" return BREAK;

">=" return GE;
"<=" return LE;
"==" return EQ;
"!=" return NE;
">" return GT;
"<" return LT;

"{" return LBRACE;
"}" return RBRACE;
"(" return LP;
")" return RP;
":" return COLON;
";" return SEMI;
"=" return ASSIGN;
"+" return PLUS;
"-" return MINUS;

[0-9]+  return NUM;
[a-zA-Z]  return ID;
[ \t\n]+  ;
.         ;

%%

int yywrap()
{
  return 1;
}

Overwriting branch.l


In [10]:
%%writefile branch.y
%{
  #include<stdio.h>
  int yylex();
  void yyerror(char *);
%}

%token IF ELSE SWITCH CASE DEFAULT BREAK
%token ID NUM
%token GT LT GE LE EQ NE
%token LBRACE RBRACE LP RP SEMI COLON ASSIGN PLUS MINUS

%%
program : stmt_list {printf("\ninput accepted\n"); };
stmt_list:stmt_list stmt
          | stmt;

stmt:if_stmt
    |switch_stmt
    |assign
    |block;
if_stmt: IF LP cond RP stmt ELSE stmt
        |IF LP cond RP stmt;

switch_stmt:SWITCH LP ID RP LBRACE case_list default_opt RBRACE;

case_list:case_list case_stmt
          |case_stmt;
case_stmt:CASE NUM COLON stmt_list BREAK SEMI;

default_opt:DEFAULT COLON stmt_list
            |;
block:LBRACE stmt_list RBRACE;
assign:ID ASSIGN expr SEMI;
expr:ID
    |NUM
    |ID PLUS ID
    |ID MINUS ID;
cond:ID relop ID;
relop:GT|LT|GE|LE|EQ|NE;
%%
void yyerror(char *s)
{
  printf("\ninvalid input\n");
}
int main()
{
  printf("enter the expression:\n");
  yyparse();
  return 0;
}


Overwriting branch.y


In [11]:
!apt-get update
!apt-get install flex bison gcc -y
!flex branch.l
!bison -dy branch.y
!gcc lex.yy.c y.tab.c -o parser

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bison is already the newest version (2:3.8.2+dfsg-1build1).
flex is already the newest version

In [14]:
%%writefile test.txt
if(a>b)
{
a=b+c;
}
else
{
a=b-c;
}

Writing test.txt


In [15]:
!./parser < test.txt

enter the expression:

input accepted


## Lex and Yacc combined

In [ ]:
#@title Writing YACC program
%%writefile program.y

%{
    #include<stdio.h>
    #include<stdlib.h>
%}
%token DIGIT LETTER UND NL
%%
stmt: variable NL {printf("Valid Identifier\n");exit(0);}
variable: LETTER alphanumeric;
alphanumeric: LETTER alphanumeric
            | DIGIT alphanumeric
            | UND alphanumeric
            | LETTER
            | DIGIT
            | UND;
%%

int yyerror(){
    printf("Invalid Identifier\n");
    exit(0);
}

void main(){
    printf("Enter the variable name: ");
    yyparse();
}

Overwriting program.y


In [ ]:
#@title Writing Lex program
%%writefile program.l

%{
    #include "y.tab.h"
%}
%%
[a-zA-Z] {return LETTER;}
[0-9] {return DIGIT;}
[_] {return UND;}
\n {return NL;}
. {return yytext[0];}
%%

Overwriting program.l


if you want to use at txt as an input

In [ ]:
%%writefile program.txt

This is a sample file.

In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

yacc -d program.y
lex program.l
cc y.tab.c lex.yy.c -ll
./a.out

y.tab.c: In function ‘yyparse’:
y.tab.c:1121:16: warning: implicit declaration of function ‘yylex’ [-Wimplicit-function-declaration]
       yychar = yylex ();
                ^~~~~
y.tab.c:1256:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? [-Wimplicit-function-declaration]
       yyerror (YY_("syntax error"));
       ^~~~~~~
       yyerrok
Enter the variable name: variable_name
Valid Identifier
